In [ ]:
import os
import shutil

# Copy dataset from Kaggle input to working folder
src = '/kaggle/input/fer2013-dataset'
dst = '/kaggle/working/fer2013-dataset-copy'

if not os.path.exists(dst):
    shutil.copytree(src, dst)

# Keep only emotions of interest
keep_classes = ['angry', 'happy', 'sad', 'surprise', 'neutral']

def clean_folders(base_dir, keep):
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if os.path.isdir(folder_path) and folder not in keep:
            shutil.rmtree(folder_path)

clean_folders(os.path.join(dst, 'train'), keep_classes)
clean_folders(os.path.join(dst, 'test'), keep_classes)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

base_dir = '/kaggle/working/fer2013-dataset-copy'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

emotions = ['angry', 'happy', 'sad', 'surprise', 'neutral']
img_size = (224, 224)  # Match MobileNetV2 pretrained weights input size
batch_size = 32

train_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_data = train_gen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size,
    class_mode='categorical', subset='training', shuffle=True, classes=emotions
)

val_data = train_gen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size,
    class_mode='categorical', subset='validation', shuffle=False, classes=emotions
)

test_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

test_data = test_gen.flow_from_directory(
    test_dir, target_size=img_size, batch_size=batch_size,
    class_mode='categorical', shuffle=False, classes=emotions
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Multiply, Reshape, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

def se_block(input_tensor, ratio=16):
    channel_axis = -1
    filters = input_tensor.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='selu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    x = Multiply()([input_tensor, se])
    return x

img_size = (224, 224)
emotions = ['angry', 'happy', 'sad', 'surprise', 'neutral']

inputs = Input(shape=(*img_size, 3))
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = se_block(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = Activation('selu')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), padding='same')(x)
x = Activation('selu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='selu')(x)
outputs = Dense(len(emotions), activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6)

# Train with frozen base model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=[early_stopping, reduce_lr]
)

# Unfreeze last 60 layers (for stronger fine-tuning)
for layer in model.layers[-60:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

history_finetune = model.fit(
    train_data,
    validation_data=val_data,
    epochs=40,
    callbacks=[early_stopping, reduce_lr]
)

# Save the model after training
model.save('fer2013_mobilenetv2_se_selu_finetuned.keras')


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load model if starting fresh
from tensorflow.keras.models import load_model
model = load_model('fer2013_mobilenetv2_se_selu_finetuned.keras')

# Evaluate on validation and test sets
val_loss, val_acc = model.evaluate(val_data)
test_loss, test_acc = model.evaluate(test_data)
print(f"Validation Accuracy: {val_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Predictions for detailed metrics
test_data.reset()
y_true = test_data.classes
y_pred_probs = model.predict(test_data)
y_pred = np.argmax(y_pred_probs, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_data.class_indices.keys()))

# Confusion matrix plot
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_data.class_indices.keys(),
            yticklabels=test_data.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for FER2013 Test Set')
plt.show()
